## This notebook trying to invest the Kaggle dataset fashion mnist

### Import Dog Dataset

In the code cell below, I import a dataset of fashion-mnist images.  We populate a few variables through the use of the `mnist_reader`

In [2]:
import numpy as np
from keras.utils import to_categorical

import sys
sys.path.insert(0,'./fashion-mnist/utils')
import mnist_reader


X_train, y_train = mnist_reader.load_mnist(\
	'./fashion-mnist/data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist(\
	'./fashion-mnist/data/fashion', kind='t10k')

print('There are total %d Train Images' % len(y_train))
print('There are total %d Testing Images' % len(y_test))
print('The size of the image is %d pixels' % X_train[0].shape[0])

There are total 60000 Train Images
There are total 10000 Testing Images
The size of the image is 784 pixels


### Pre-process the Data
When using TensorFlow as backend, Keras CNNs require a 4D array (which we'll also refer to as a 4D tensor) as input, with shape
$$
(\text{nb_samples}, \text{rows}, \text{columns}, \text{channels}),
$$
where nb_samples corresponds to the total number of images (or samples), and rows, columns, and channels correspond to the number of rows, columns, and channels for each image, respectively.
The path_to_tensor function below takes a string-valued file path to a color image as input and returns a 4D tensor suitable for supplying to a Keras CNN. The function first loads the image and resizes it to a square image that is $28 \times 28$ pixels. Next, the image is converted to an array, which is then resized to a 4D tensor. In this case, since we are working with gray images, each image has one channel. Likewise, since we are processing total images (or sample), the returned tensor will always have shape
$$
(60000, 224, 224, 1).
$$

The rescale the images by dividing every pixel in every image by 255.

In [3]:
y_train = to_categorical(y_train)
print (y_train.shape) # (n_samples, 10)

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
print("train set: ", X_train.shape) # (n_samples, 28, 28, 1)
print("test set: ", X_test.shape) 

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# print (X_train[0])

(60000, 10)
train set:  (60000, 28, 28, 1)
test set:  (10000, 28, 28, 1)


### Cross Validation
Using 30% of the training data as a validation set to optimizd the classifier

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X_train,\
                                                                y_train,\
                                                               test_size=0.3,\
                                                               random_state=123)
# print(y_validation[0])

[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]


### Creating Model Architecture from scratch

Create a CNN to classify dog breed.  At the end of your code cell block, summarize the layers of your model by executing the line:
    
        model.summary()


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, GlobalAveragePooling2D
from keras.layers import Flatten, Conv2D, MaxPooling2D

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, padding='same',\
                 input_shape = X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv2D(filters=128, kernel_size=3, padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# model.summary()

model.add(GlobalAveragePooling2D())
model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
activation_30 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_31 (Activation)   (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 7, 7, 64)          0         
__________

### Compile the Model

In [20]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',\
              metrics=['accuracy'])

### Train the Model
Use the model checkpointing to save the model that attains the best validation loss.

In [38]:
from keras.callbacks import ModelCheckpoint

# epochs = 5

checkpointer = ModelCheckpoint(filepath='./saved_model/weights.best.from_scrathc.hdf5',\
                              verbose=1, save_best_only=True)
model.fit(X_train, y_train, validation_data=(X_validation, y_validation),\
         epochs=50, batch_size=200, callbacks=[checkpointer], verbose=2)

Train on 10084 samples, validate on 4322 samples
Epoch 1/50
Epoch 00000: val_loss improved from inf to 1.07303, saving model to ./saved_model/weights.best.from_scrathc.hdf5
24s - loss: 1.0948 - acc: 0.5625 - val_loss: 1.0730 - val_acc: 0.5821
Epoch 2/50
Epoch 00001: val_loss improved from 1.07303 to 0.92692, saving model to ./saved_model/weights.best.from_scrathc.hdf5
24s - loss: 1.0337 - acc: 0.5952 - val_loss: 0.9269 - val_acc: 0.6798
Epoch 3/50
Epoch 00002: val_loss did not improve
26s - loss: 0.9642 - acc: 0.6285 - val_loss: 0.9533 - val_acc: 0.6400
Epoch 4/50
Epoch 00003: val_loss improved from 0.92692 to 0.88057, saving model to ./saved_model/weights.best.from_scrathc.hdf5
27s - loss: 0.9263 - acc: 0.6474 - val_loss: 0.8806 - val_acc: 0.6772
Epoch 5/50
Epoch 00004: val_loss improved from 0.88057 to 0.77735, saving model to ./saved_model/weights.best.from_scrathc.hdf5
26s - loss: 0.8827 - acc: 0.6694 - val_loss: 0.7773 - val_acc: 0.7233
Epoch 6/50
Epoch 00005: val_loss improved fr

### Load the Model with the Best Validation Loss

In [39]:
model.load_weights('./saved_model/weights.best.from_scrathc.hdf5')

### Test the Model
Try out the model on the test dataset of fashion-mnist images

In [40]:
predictions = [np.argmax(model.predict(np.expand_dims(test, axis=0))) for test in X_test]

# report test accuracy
# test_accuracy = 100 * np.sum(np.array(predictions) == np.argmax(y_test,\
#                                                     axis=0)) / len(predictions)
# print("Test accuracy: %.4f%%" % test_accuracy)
# score = model.evaluate(X_test, y_test, verbose=0)

# print('Test loss: ', score[0])
# print('Test accuracy', score[1])

In [41]:
# report test accuracy
test_accuracy = 100 * np.sum(np.array(predictions) == np.argmax(y_test,\
                                                    axis=0)) / len(predictions)
print("Test accuracy: %.4f%%" % test_accuracy)

Test accuracy: 11.5500%
